In [1]:
import time
import random
import tqdm


import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from geopy import distance
from geopy.extra.rate_limiter import RateLimiter

pd.set_option('max_colwidth', 600)
pd.set_option('display.width', 600)

In [2]:
df = pd.read_csv('df_2.csv', index_col=0)

In [3]:
df['date'] = df['text'].apply(lambda x: x.split('.  Продам')[0])
df['note_len'] = df['text'].apply(lambda x: len(x))
df = df.loc[df['note_len'] > 3]

df['date'] = pd.to_datetime(df['date'], format="%d.%m.%Y")

df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['dayofweek'] = df['date'].dt.dayofweek #Monday=0, Sunday=6

In [4]:
df['rooms'] = df['text'].apply(lambda x: x.split("Продам ")[1].split('квартиру')[0])

In [5]:
def district_sort(text):
    text = text.lower()
    if 'орджоникидз' in text:
        if 'левый берег' in text or 'левом' in text:
            return 'Орджоникидзевский(левый берег)'
        else:
            return 'Орджоникидзевский'
    elif 'правобереж' in text:
        return 'Правобережный'
    elif 'ленинск' in text:
        if 'левый берег' in text or 'левом' in text:
            return 'Ленинский(левый берег)'
        else:
            return 'Ленинский'

In [6]:
df['district'] = df['text'].apply(district_sort)
df['text'] = df['text'].map(lambda x: x.replace('ул. ул. ', 'ул. '))
df['street'] = df['text'].apply(lambda x: x.split("ул.")[1].split(', этаж')[0] if "ул." in x else 'unknown')
df['street'] = df['street'].map(lambda x: x.replace('ул ', ''))

df['street'] = df['street'].apply(lambda x: x.lstrip(". ").rstrip(", ")).map(lambda x: x.replace('дом ', ''))

In [7]:
df[['floor', 'total_floors']] = df['text'].apply(
    lambda x: x.split("этаж")[1].split(',')[0].split('.')[0] if "этаж" in x else 'unknown').str.split('/', expand=True)

In [8]:
df['total_area'] = df['text'].apply(lambda x: x.split("Общая площадь - ")[1].split(' кв.м.')[0] if "Общая площадь" in x else 'unknown')
df['living_area'] = df['text'].apply(lambda x: x.split("жилая площадь - ")[1].split(' кв.м.')[0] if "жилая площадь - " in x else 'unknown')
df['kitchen_area'] = df['text'].apply(lambda x: x.split("кухня - ")[1].split(' кв.м.')[0] if "кухня - " in x else 'unknown')

df['price'] = df['text'].apply(lambda x: x.split("Цена - ")[1].split(' т.р.')[0] if "Цена - " in x else None)

In [9]:
df['street'] = 'Магнитогорск, ' + df['street']

In [10]:
from tqdm import tqdm
tqdm.pandas()

geolocator = Nominatim(user_agent="my-app")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2)

df['location'] = df['street'].progress_apply(geocode)

 99%|█████████▊| 461/467 [15:22<00:11,  1.99s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('Магнитогорск, Западное шоссе, 95',), **{}).
Traceback (most recent call last):
  File "e:\GITlocal\Magnitogorsk_real_estate\.venv\lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
  File "e:\GITlocal\Magnitogorsk_real_estate\.venv\lib\site-packages\urllib3\connection.py", line 461, in getresponse
    httplib_response = super().getresponse()
  File "C:\Users\s7\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "C:\Users\s7\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "C:\Users\s7\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\s7\AppData\Local\Prog

In [11]:
df['lat_long'] = df['location'].dropna().apply(lambda x: x[1])

In [12]:
df.loc[df['district'].isna(), 'district'] = df['location'].dropna().apply(lambda x: x[0].split('район')[0].split(', ')[-1] if 'район' in str(x) else 'unknown')
parks_and_center = {'park_eko':(53.402233, 58.952641),
'attraction':(53.381266, 58.953624),
'south':(53.351887, 58.990688),
'eternal_fire':(53.405914, 58.991158),
'veter':(53.429143, 59.000213),
'city_center':(53.407164, 58.980285)}

# Итерирование по ключам словаря parks
for key in parks_and_center:
    # Создание нового столбца с именем ключа и заполнение его значениями
    df[key] = [(parks_and_center[key])]* len(df['lat_long'])
    df[key] = df[[key, 'lat_long']].dropna().apply(lambda row: round(distance.distance(row[key], row["lat_long"]).km, 2), axis=1)
df.info()
df['rooms'].value_counts()
df['rooms'][0]
def rooms_num(rooms):
    rooms = rooms.rstrip()
    answ = -1
    if rooms == 'однокомнатную':
        answ = 1
    if rooms == 'двухкомнатную':
        answ = 2
    if rooms == 'трехкомнатную':
        answ = 3
    if rooms == 'четырехкомнатную':
        answ = 4
    if rooms == 'многокомнатную':
        answ = 5
    return answ
df['rooms'] = df['rooms'].apply(rooms_num)
df['rooms']
df['district'].value_counts()
df.sample(2)

df.info()
df[['floor', 'total_floors', 'total_area', 'living_area', 'kitchen_area', 'price']] = \
df[['floor', 'total_floors', 'total_area', 'living_area', 'kitchen_area', 'price']].apply(pd.to_numeric, errors='coerce').dropna()
df['district'] = df['district'].dropna().apply(lambda x: x.lstrip().rstrip())
df = df.drop(['text',  'note_len', 'location'], axis=1)
df['price'] = df['price'].dropna()
df = df.dropna(subset=['price'])
df.select_dtypes('number').corr()
df['price_sq_meter'] = df['price'] / df['total_area']
df['floor_type'] = 'middle'
df.loc[df.floor == 1, 'floor_type'] = 'first'
df.loc[(df.floor == df.total_floors) & (df.floor != 1), 'floor_type'] = 'last'
df.to_csv('df_3.csv') #ready for analysis